In [1]:
import tensorflow as tf
from tensorflow.keras import optimizers, losses
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from tensorflow.keras.callbacks import EarlyStopping

2023-12-06 10:03:56.109891: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-06 10:03:56.189724: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-06 10:03:56.624890: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 10:03:56.624967: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 10:03:56.754251: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
IM_MODEL_NAME = "distilbert-base-uncased"
IM_BATCH_SIZE = 2
IM_LEARNING_RATE = 3e-5
IM_TOKEN_MAX_LEN = 50   ### currently set at 50 to speed up basic model training
IM_TEST_SPLIT = 0.2
IM_VALIDATION_SPLIT = 0.3   ### refers to split withing training data (not whole dataset)
IM_EPOCHS = 5   ### currently set to 5 to speed up basic model training
IM_PATIENCE = 2   ### currently set to 2 due to the low number of epochs (5)


In [3]:
import pandas as pd


In [ ]:
df = pd.read_csv('/home/connor/code/zulu-tango/news_and_echo_bubbles/raw_data/cleaned.csv', index_col=0)


In [ ]:
# drop irrelevant columns and take a sample of 50 to test
df = df.drop(columns = ["link", "pdate", "title", "author", "text", "keywords", "tags", "compound",\
                            "neg", "neu", "pos", "polarity", "subjectivity", "time", "urls"]).sample(50)


In [ ]:
df.head()


In [ ]:
continuous_list = [-1.89, -1.02, 1.6, 3.65, 2.8, -2.47, 1.39, 2.78, -3.65, -3.4, -3.48, 2.44, -4.07, -4.28, 4.71, 3.03, -0.68, 3.05, 0.06,\
                   4.86, 0.31, 2.18, -0.31, -3.47, -1.99, 1.81, -0.91, -1.17, 3.34, 1.41, -2.23, -3.17, 2.58, -3.13, -1.23, 0.56, 4.48, 3.48,\
                   -2.48, 3.37, -4.81, -1.84, -0.42, 4.94, 4.14, -1.59, 0.16, -4.29, -4.24, -4.23]


In [ ]:
len(continuous_list)


In [ ]:
discrete_list = [-1, -2, 2, 0, 2, 0, 1, 0, 0, 2, 2, 1, 2, 0, -1, 2, -2, -2, -1, -2, 2, 1, 2, 1, 1, -1, -2, 2, 1, -2, -2, 2, -1, 0, 0,\
                 -1, -1, -2, 0, 1, -1, 1, 2, -1, 0, 1, 2, 2, 0, 0]


In [ ]:
len(discrete_list)


In [ ]:
df["continuous_bias_score"] = continuous_list
df["discrete_bias_score"] = discrete_list
df.head()


In [ ]:
### TO DO: MAKE THE MODEL WORK USING THE "CONTINUOUS_BIAS_SCORE" AND "DISCRETE_BIAS_SCORE" COLUMNS
### THE OUTPUT SHOULD STILL BE A PROBABILITY FROM 0 TO 1


In [ ]:
from tensorflow.keras.utils import to_categorical


In [ ]:
one_hot_dict = {-2:[1,0,0,0,0],
                -1:[0,1,0,0,0],
                0:[0,0,1,0,0],
                1:[0,0,0,1,0],
                2:[0,0,0,0,1]}


In [ ]:
df["one_hot_discrete"] = df["discrete_bias_score"].map(one_hot_dict)


In [ ]:
df.head()


In [ ]:
def NEW_get_X_and_y(df):
    """
    Gets from our dataset: (i) the feature (i.e. X - the pre-processed text);
    and (ii) the target (i.e. y - the ideology: left wing = 0 / right wing = 1).
    These need to be converted into lists for use in our model.
    """

    X = df["pre_process_text"].tolist()
    y = df["one_hot_discrete"].tolist()

    return X, y


In [ ]:
X, y = NEW_get_X_and_y(df)


In [ ]:
tokenizer = instantiate_tokenizer()


In [ ]:
tokens = text_tokenizer(X, tokenizer)


In [ ]:
tfdataset = tf_dataset_constructor(tokens, y)


In [ ]:
tfdataset_train, tfdataset_val, tfdataset_test = train_test_split(X, tfdataset)


In [ ]:
def NEW_ideology_model(tfdataset_train,
                   tfdataset_val,
                   model_name = IM_MODEL_NAME,
                   learning_rate = IM_LEARNING_RATE,
                   batch_size = IM_BATCH_SIZE,
                   epochs = IM_EPOCHS,
                   patience = IM_PATIENCE):

    """
    Set up an run a DistilBert model on our TensorFlow training dataset.
    """

    num_labels = 5  ################

    # set up model
    model = TFDistilBertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    # define loss function
    loss = losses.BinaryCrossentropy(from_logits=True)

    # define optimizer to be used to minimise loss
    optimizer = optimizers.Adam(learning_rate)

    # compile model
    model.compile(optimizer = optimizer,
                  loss = loss,
                  metrics = "accuracy")

    # fit model
    model.fit(tfdataset_train,
              batch_size = batch_size,
              epochs = epochs,
              validation_data = tfdataset_val,
              callbacks = EarlyStopping(patience = patience, restore_best_weights = True))

    return model


In [ ]:
model = NEW_ideology_model(tfdataset_train,
                   tfdataset_val)


In [ ]:
accuracy = ideology_model_evaluator(model, tfdataset_test)


In [ ]:
pred_probas = ideology_model_predictor(model,tokens)


In [ ]:
pred_probas


In [ ]:
import numpy as np


In [ ]:
top_class_list = []

for row in range(len(pred_probas)):

    conversion_dict = {0 : "left",
                       1 : "leans left",
                       2 : "centre",
                       3 : "leans right",
                       4 : "right"}

    top_class_list.append(conversion_dict[np.argmax(pred_probas[row])])

print(top_class_list)


## TESTING IN FULL!!!!

In [4]:
df = pd.read_csv('/home/connor/code/zulu-tango/news_and_echo_bubbles/raw_data/cleaned.csv', index_col=0)


In [5]:
df = df.drop(columns = ["link", "classifier", "title", "author", "text", "keywords", "tags", "compound",\
                            "neg", "neu", "pos", "polarity", "subjectivity", "time"]).sample(50)


In [6]:
df.head()


,pdate,urls,pre_process_text
1071,2021-03-15,www.newyorker.com,let our journalists help you make sense of the...
1099,2021-03-02,www.motherjones.com,let our journalists help you make sense of the...
1413,2021-03-22,www.huffpost.com,on march a white gunman killed eight people ...
410,2020-01-26,www.cnn.com,the populist wave in politics on both sides of...
1061,2021-01-21,www.newyorker.com,let our journalists help you make sense of the...


In [7]:
discrete_list = [-1, -2, 2, 0, 2, 0, 1, 0, 0, 2, 2, 1, 2, 0, -1, 2, -2, -2, -1, -2, 2, 1, 2, 1, 1, -1, -2, 2, 1, -2, -2, 2, -1, 0, 0,\
                 -1, -1, -2, 0, 1, -1, 1, 2, -1, 0, 1, 2, 2, 0, 0]


In [8]:
df["discrete_bias_score"] = discrete_list
df.head()


,pdate,urls,pre_process_text,discrete_bias_score
1071,2021-03-15,www.newyorker.com,let our journalists help you make sense of the...,-1
1099,2021-03-02,www.motherjones.com,let our journalists help you make sense of the...,-2
1413,2021-03-22,www.huffpost.com,on march a white gunman killed eight people ...,2
410,2020-01-26,www.cnn.com,the populist wave in politics on both sides of...,0
1061,2021-01-21,www.newyorker.com,let our journalists help you make sense of the...,2


In [9]:
df["5_step_classifier"] = df["discrete_bias_score"]


In [10]:
df.head()


,pdate,urls,pre_process_text,discrete_bias_score,5_step_classifier
1071,2021-03-15,www.newyorker.com,let our journalists help you make sense of the...,-1,-1
1099,2021-03-02,www.motherjones.com,let our journalists help you make sense of the...,-2,-2
1413,2021-03-22,www.huffpost.com,on march a white gunman killed eight people ...,2,2
410,2020-01-26,www.cnn.com,the populist wave in politics on both sides of...,0,0
1061,2021-01-21,www.newyorker.com,let our journalists help you make sense of the...,2,2


In [11]:
df= df.drop(columns = "discrete_bias_score")


In [12]:
df.head()


,pdate,urls,pre_process_text,5_step_classifier
1071,2021-03-15,www.newyorker.com,let our journalists help you make sense of the...,-1
1099,2021-03-02,www.motherjones.com,let our journalists help you make sense of the...,-2
1413,2021-03-22,www.huffpost.com,on march a white gunman killed eight people ...,2
410,2020-01-26,www.cnn.com,the populist wave in politics on both sides of...,0
1061,2021-01-21,www.newyorker.com,let our journalists help you make sense of the...,2


In [13]:
import numpy as np


In [14]:
df = df.reset_index()

In [15]:
df.head()

,index,pdate,urls,pre_process_text,5_step_classifier
0,1071,2021-03-15,www.newyorker.com,let our journalists help you make sense of the...,-1
1,1099,2021-03-02,www.motherjones.com,let our journalists help you make sense of the...,-2
2,1413,2021-03-22,www.huffpost.com,on march a white gunman killed eight people ...,2
3,410,2020-01-26,www.cnn.com,the populist wave in politics on both sides of...,0
4,1061,2021-01-21,www.newyorker.com,let our journalists help you make sense of the...,2


In [16]:
n = 5

three_hot_dict = {-1 : [1,0,0],
                  0 : [0,1,0],
                  1 : [0,0,1]}

five_hot_dict = {-2 : [1,0,0,0,0],
                 -1 : [0,1,0,0,0],
                 0 : [0,0,1,0,0],
                 1 : [0,0,0,1,0],
                 2 : [0,0,0,0,1]}


In [17]:
def bias_score_encoding(df, n):

    if n == 5:
        df["one_hot_discrete"] = df["5_step_classifier"].map(five_hot_dict)

    if n == 3:
        df["one_hot_discrete"] = df["3_step_classifier"].map(three_hot_dict)

    return df


In [ ]:
df = bias_score_encoding(df,5)


In [18]:
def n_class_get_X_and_y(df):
    """
    xxxxxxxxxx
    """

    X = df["pre_process_text"].tolist()
    y = df["one_hot_discrete"].tolist()

    return X, y


In [ ]:
X, y = n_class_get_X_and_y(df)


In [19]:
def instantiate_tokenizer(model_name = IM_MODEL_NAME):
    """
    Define the tokenizer we want to use in our modelling.
    """

    tokenizer = DistilBertTokenizer.from_pretrained(model_name)

    return tokenizer


In [ ]:
tokenizer = instantiate_tokenizer()


In [20]:
def text_tokenizer(X,
                   tokenizer,
                   max_len = IM_TOKEN_MAX_LEN,
                   truncation = True,
                   padding = "max_length"):
    """
    Returns a dictionary of tokenized text with 2 keys: "input_ids" and "attention_mask".
    These 2 keys are required for the input into the DistilBert model.
    """

    tokens = tokenizer(X, max_length = max_len, truncation = truncation, padding = padding)

    return tokens


In [ ]:
tokens = text_tokenizer(X, tokenizer)


In [21]:
def tf_dataset_constructor(tokens,
                           y):
    """
    Using the tokenized input from the text_tokenizer function,
    returns TensorFlow objects for use in the DistilBert model.
    """

    tfdataset = tf.data.Dataset.from_tensor_slices((dict(tokens),y))

    return tfdataset


In [ ]:
tfdataset = tf_dataset_constructor(tokens, y)


In [22]:
def train_test_split(X,
                     tfdataset,
                     test_split = IM_TEST_SPLIT,
                     val_split = IM_VALIDATION_SPLIT,
                     batch_size = IM_BATCH_SIZE):
    """
    This function splits the TensorFlow object created in the tf_dataset_constructor function
    into train, valdiation and test sets.
    """

    # get the sizes of the train and validation sets
    train_size = int(len(X) * (1-test_split))
    val_size = int(train_size * val_split)

    # shuffle the full dataset
    tfdataset = tfdataset.shuffle(len(X))

    # from the full datset, get out the train, validation and test sets
    tfdataset_train = tfdataset.take(train_size)
    tfdataset_val = tfdataset.skip(train_size - val_size).take(val_size)
    tfdataset_test = tfdataset.skip(train_size)

    # batch the train, validation and test sets
    tfdataset_train = tfdataset_train.batch(batch_size)
    tfdataset_val = tfdataset_val.batch(batch_size)
    tfdataset_test = tfdataset_test.batch(batch_size)

    return tfdataset_train, tfdataset_val, tfdataset_test


In [ ]:
tfdataset_train, tfdataset_val, tfdataset_test = train_test_split(X, tfdataset)


In [23]:
def n_class_ideology_model(tfdataset_train,
                           tfdataset_val,
                           n,
                           model_name = IM_MODEL_NAME,
                           learning_rate = IM_LEARNING_RATE,
                           batch_size = IM_BATCH_SIZE,
                           epochs = IM_EPOCHS,
                           patience = IM_PATIENCE):

    """
    Set up an run a DistilBert model on our TensorFlow training dataset.
    """

    # set up model
    model = TFDistilBertForSequenceClassification.from_pretrained(model_name, num_labels = n)

    # define loss function
    loss = losses.BinaryCrossentropy(from_logits=True)

    # define optimizer to be used to minimise loss
    optimizer = optimizers.Adam(learning_rate)

    # compile model
    model.compile(optimizer = optimizer,
                  loss = loss,
                  metrics = "accuracy")

    # fit model
    model.fit(tfdataset_train,
              batch_size = batch_size,
              epochs = epochs,
              validation_data = tfdataset_val,
              callbacks = EarlyStopping(patience = patience, restore_best_weights = True))

    return model


In [ ]:
model = n_class_ideology_model(tfdataset_train,tfdataset_val,5)


In [24]:
def ideology_model_predictor(model,
                             tokens):
    """
    This function uses the model output from the ideology_model function to output the
    probabilities of each individual article being left or right wing (0 = left wing,
    1 = right wing). As the model spits out log odds rather than probabilities, these
    also need to be converted in this function into probabilities.
    """

    # firstly create a TensorFlow version of our tokenized dataset without our y
    tfdataset_no_y = tf.data.Dataset.from_tensor_slices(dict(tokens))

    # use this to get out the logits for our model
    pred_logits = model.predict(tfdataset_no_y)[0]

    # convert these into probabilties
    pred_probas = tf.nn.softmax(pred_logits).numpy()

    return pred_probas


In [ ]:
pred_probas = ideology_model_predictor(model, tokens)
pred_probas


In [25]:
def top_class(pred_probas):

    top_class_list = []

    for row in range(len(pred_probas)):

        conversion_dict = {0 : "left",
                       1 : "leans left",
                       2 : "centre",
                       3 : "leans right",
                       4 : "right"}

        top_class_list.append(conversion_dict[np.argmax(pred_probas[row])])

    return top_class_list


In [ ]:
top_class_list = top_class(pred_probas)
top_class_list


In [26]:
def top_class_probas(pred_probas):

    top_class_probas_list = []

    for row in range(len(pred_probas)):
        top_class_probas_list.append(max(pred_probas[row]))

    return top_class_probas_list


In [ ]:
top_class_probas_list = top_class_probas(pred_probas)
top_class_probas_list


In [27]:
def top_class_certainty(top_class_probas_list):

    top_class_certainty_list = []

    for prob in top_class_probas_list:

        if prob > 0.8:
            top_class_certainty_list.append("highly certain")

        elif prob > 0.6:
            top_class_certainty_list.append("certain")

        elif prob > 0.4:
            top_class_certainty_list.append("fairly certain")

        else:
            top_class_certainty_list.append("low certainty")


    return top_class_certainty_list


In [ ]:
top_class_certainty_list = top_class_certainty(top_class_probas_list)
top_class_certainty_list


In [28]:
def full_n_class_ideology_model(df, n):

    df = bias_score_encoding(df, n)

    X, y = n_class_get_X_and_y(df)

    tokenizer = instantiate_tokenizer(model_name = IM_MODEL_NAME)

    tokens = text_tokenizer(X,
                            tokenizer,
                            max_len = IM_TOKEN_MAX_LEN,
                            truncation = True,
                            padding = "max_length")

    tfdataset = tf_dataset_constructor(tokens, y)

    # the following function automatically returns the test dataset, even though this is
    # not used further, as we do not evaluate the model accuracy within this function.

    tfdataset_train, tfdataset_val, tfdataset_test =\
    train_test_split(X,
                    tfdataset,
                    test_split = IM_TEST_SPLIT,
                    val_split = IM_VALIDATION_SPLIT,
                    batch_size = IM_BATCH_SIZE)

    model = n_class_ideology_model(tfdataset_train,
                           tfdataset_val,
                           n,
                           model_name = IM_MODEL_NAME,
                           learning_rate = IM_LEARNING_RATE,
                           batch_size = IM_BATCH_SIZE,
                           epochs = IM_EPOCHS,
                           patience = IM_PATIENCE)

    pred_probas = ideology_model_predictor(model, tokens)

    top_class_list = top_class(pred_probas)
    top_class_probas_list = top_class_probas(pred_probas)
    top_class_certainty_list = top_class_certainty(top_class_probas_list)

    df["pred_class"] = top_class_list
    df["pred_class_proba"] = top_class_probas_list
    df["pred_class_certainty"] = top_class_certainty_list

    return df


In [ ]:
output_df = full_n_class_ideology_model(df,5)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/5
 1/20 [>.............................] - ETA: 4:21 - loss: 0.6801 - accuracy: 0.5000

In [ ]:
output_df


In [1]:
uk_us_dict = {"msnbc.com" : "US",
"theatlantic.com" : "US",
"alternet.org" : "US",
"thedailybeast.com" : "US",
"huffpost.com" : "US",
"theintercept.com" : "US",
"jacobinmag.com" : "US",
"democracynow.org" : "US",
"motherjones.com" : "US",
"newyorker.com" : "US",
"thenation.com" : "US",
"slate.com" : "US",
"vox.com" : "US",
"yahoo.com" : "US",
"abcnews.go.com" : "US",
"theguardian.com" : "UK",
"bloomberg.com" : "US",
"time.com" : "US",
"npr.org" : "US",
"propublica.org" : "US",
"insider.com" : "US",
"nbcnews.com" : "US",
"nytimes.com" : "US",
"axios.com" : "US",
"cbsnews.com" : "US",
"apnews.com" : "US",
"cnn.com" : "US",
"washingtonpost.com" : "US",
"usatoday.com" : "US",
"politico.com" : "US",
"bbc.com" : "UK",
"csmonitor.com" : "US",
"forbes.com" : "US",
"thehill.com" : "US",
"reuters.com" : "UK",
"newsweek.com" : "US",
"marketwatch.com" : "US",
"newsnationnow.com" : "US",
"realclearpolitics.com" : "US",
"wsj.com" : "US",
"thedispatch.com" : "US",
"theepochtimes.com" : "US",
"nypost.com" : "US",
"nationalreview.com" : "US",
"foxbusiness.com" : "US",
"reason.com" : "US",
"washingtonexaminer.com" : "US",
"washingtontimes.com" : "US",
"foxnews.com" : "US",
"spectator.org" : "US",
"breitbart.com" : "US",
"theamericanconservative.com" : "US",
"theblaze.com" : "US",
"dailycaller.com" : "US",
"cbn.com" : "US",
"dailywire.com" : "US",
"thepostmillennial.com" : "Canada",
"dailymail.co.uk" : "UK",
"thefederalist.com" : "US",
"ijr.com" : "US",
"newsmax.com" : "US",
"freebeacon.com" : "US",
"oann.com" : "US"}


In [2]:
list(uk_us_dict.keys())


['msnbc.com',
 'theatlantic.com',
 'alternet.org',
 'thedailybeast.com',
 'huffpost.com',
 'theintercept.com',
 'jacobinmag.com',
 'democracynow.org',
 'motherjones.com',
 'newyorker.com',
 'thenation.com',
 'slate.com',
 'vox.com',
 'yahoo.com',
 'abcnews.go.com',
 'theguardian.com',
 'bloomberg.com',
 'time.com',
 'npr.org',
 'propublica.org',
 'insider.com',
 'nbcnews.com',
 'nytimes.com',
 'axios.com',
 'cbsnews.com',
 'apnews.com',
 'cnn.com',
 'washingtonpost.com',
 'usatoday.com',
 'politico.com',
 'bbc.com',
 'csmonitor.com',
 'forbes.com',
 'thehill.com',
 'reuters.com',
 'newsweek.com',
 'marketwatch.com',
 'newsnationnow.com',
 'realclearpolitics.com',
 'wsj.com',
 'thedispatch.com',
 'theepochtimes.com',
 'nypost.com',
 'nationalreview.com',
 'foxbusiness.com',
 'reason.com',
 'washingtonexaminer.com',
 'washingtontimes.com',
 'foxnews.com',
 'spectator.org',
 'breitbart.com',
 'theamericanconservative.com',
 'theblaze.com',
 'dailycaller.com',
 'cbn.com',
 'dailywire.com',

In [3]:
list(df["urls"])[17][4:]


NameError: name 'df' is not defined

In [ ]:
uk_us_dict[list(df["urls"])[0][4:]]


In [ ]:
def add_country_source(df):

    country_list = []

    for i in range(len(df["urls"])):

        if list(df["urls"])[i] in list(uk_us_dict.keys()):
            country_list.append(uk_us_dict[list(df["urls"])[i]])

        # removing "www." from the beginnging of URLs if present
        elif list(df["urls"])[i][4:] in list(uk_us_dict.keys()):
            country_list.append(uk_us_dict[list(df["urls"])[i][4:]])

        # removing "www1." or "www2." from the beginnging of URLs if present
        elif list(df["urls"])[i][5:] in list(uk_us_dict.keys()):
            country_list.append(uk_us_dict[list(df["urls"])[i][5:]])

        else:
            country_list.append("country unknown")

    df["country"] = country_list

    return df


In [ ]:
new_df = add_country_source(df)

new_df.head(20)


In [ ]:
new_df.reset_index()

In [ ]:
new_df = new_df.drop_duplicates(subset = ["urls"], keep='first', ignore_index=True)
new_df

## The below is a working model based on binary classification only

In [ ]:
IM_MODEL_NAME = "distilbert-base-uncased"
IM_BATCH_SIZE = 2
IM_LEARNING_RATE = 3e-5
IM_TOKEN_MAX_LEN = 50   ### currently set at 50 to speed up basic model training
IM_TEST_SPLIT = 0.2
IM_VALIDATION_SPLIT = 0.3   ### refers to split withing training data (not whole dataset)
IM_EPOCHS = 5   ### currently set to 5 to speed up basic model training
IM_PATIENCE = 2   ### currently set to 2 due to the low number of epochs (5)


In [ ]:
def get_X_and_y(df):
    """
    Gets from our dataset: (i) the feature (i.e. X - the pre-processed text);
    and (ii) the target (i.e. y - the ideology: left wing = 0 / right wing = 1).
    These need to be converted into lists for use in our model.
    """

    X = df["pre_process_text"].tolist()
    y = df["classifier"].tolist()

    return X, y


In [ ]:
def instantiate_tokenizer(model_name = IM_MODEL_NAME):
    """
    Define the tokenizer we want to use in our modelling.
    """

    tokenizer = DistilBertTokenizer.from_pretrained(model_name)

    return tokenizer


In [ ]:
def text_tokenizer(X,
                   tokenizer,
                   max_len = IM_TOKEN_MAX_LEN,
                   truncation = True,
                   padding = "max_length"):
    """
    Returns a dictionary of tokenized text with 2 keys: "input_ids" and "attention_mask".
    These 2 keys are required for the input into the DistilBert model.
    """

    tokens = tokenizer(X, max_length = max_len, truncation = truncation, padding = padding)

    return tokens


In [ ]:
def tf_dataset_constructor(tokens,
                           y):
    """
    Using the tokenized input from the text_tokenizer function,
    returns TensorFlow objects for use in the DistilBert model.
    """

    tfdataset = tf.data.Dataset.from_tensor_slices((dict(tokens),y))

    return tfdataset


In [ ]:
def train_test_split(X,
                     tfdataset,
                     test_split = IM_TEST_SPLIT,
                     val_split = IM_VALIDATION_SPLIT,
                     batch_size = IM_BATCH_SIZE):
    """
    This function splits the TensorFlow object created in the tf_dataset_constructor function
    into train, valdiation and test sets.
    """

    # get the sizes of the train and validation sets
    train_size = int(len(X) * (1-test_split))
    val_size = int(train_size * val_split)

    # shuffle the full dataset
    tfdataset = tfdataset.shuffle(len(X))

    # from the full datset, get out the train, validation and test sets
    tfdataset_train = tfdataset.take(train_size)
    tfdataset_val = tfdataset.skip(train_size - val_size).take(val_size)
    tfdataset_test = tfdataset.skip(train_size)

    # batch the train, validation and test sets
    tfdataset_train = tfdataset_train.batch(batch_size)
    tfdataset_val = tfdataset_val.batch(batch_size)
    tfdataset_test = tfdataset_test.batch(batch_size)

    return tfdataset_train, tfdataset_val, tfdataset_test


In [ ]:
def ideology_model(tfdataset_train,
                   tfdataset_val,
                   model_name = IM_MODEL_NAME,
                   learning_rate = IM_LEARNING_RATE,
                   batch_size = IM_BATCH_SIZE,
                   epochs = IM_EPOCHS,
                   patience = IM_PATIENCE):

    """
    Set up an run a DistilBert model on our TensorFlow training dataset.
    """

    # set up model
    model = TFDistilBertForSequenceClassification.from_pretrained(model_name)

    # define loss function
    loss = losses.SparseCategoricalCrossentropy(from_logits=True)

    # define optimizer to be used to minimise loss
    optimizer = optimizers.Adam(learning_rate)

    # compile model
    model.compile(optimizer = optimizer,
                  loss = loss,
                  metrics = "accuracy")

    # fit model
    model.fit(tfdataset_train,
              batch_size = batch_size,
              epochs = epochs,
              validation_data = tfdataset_val,
              callbacks = EarlyStopping(patience = patience, restore_best_weights = True))

    return model


In [ ]:
def ideology_model_evaluator(model,
                             tfdataset_test,
                             batch_size = IM_BATCH_SIZE):
    """
    Evaluate our model on the TensorFlow test dataset
    """

    benchmarks = model.evaluate(tfdataset_test, batch_size = batch_size, return_dict = True)
    accuracy = benchmarks["accuracy"]

    return accuracy


In [ ]:
def ideology_model_predictor(model,
                             tokens):
    """
    This function uses the model output from the ideology_model function to output the
    probabilities of each individual article being left or right wing (0 = left wing,
    1 = right wing). As the model spits out log odds rather than probabilities, these
    also need to be converted in this function into probabilities.
    """

    # firstly create a TensorFlow version of our tokenized dataset without our y
    tfdataset_no_y = tf.data.Dataset.from_tensor_slices(dict(tokens))

    # use this to get out the logits for our model
    pred_logits = model.predict(tfdataset_no_y)[0]

    # convert these into probabilties
    pred_probas = tf.nn.softmax(pred_logits).numpy()

    return pred_probas


In [ ]:
def full_ideology_model(df):
    """
    Combine all above functions into one master function, except for the
    ideology_model_evaluator function, as we do not need the accuracy output here.
    """

    X, y = get_X_and_y(df)

    tokenizer = instantiate_tokenizer(model_name = IM_MODEL_NAME)

    tokens = text_tokenizer(X,
                            tokenizer,
                            max_len = IM_TOKEN_MAX_LEN,
                            truncation = True,
                            padding = "max_length")

    tfdataset = tf_dataset_constructor(tokens, y)

    # the following function automatically returns the test dataset, even though this is
    # not used further, as we do not evaluate the model accuracy within this function.

    tfdataset_train, tfdataset_val, tfdataset_test =\
    train_test_split(X,
                    tfdataset,
                    test_split = IM_TEST_SPLIT,
                    val_split = IM_VALIDATION_SPLIT,
                    batch_size = IM_BATCH_SIZE)

    model = ideology_model(tfdataset_train,
                           tfdataset_val,
                           model_name = IM_MODEL_NAME,
                           learning_rate = IM_LEARNING_RATE,
                           batch_size = IM_BATCH_SIZE,
                           epochs = IM_EPOCHS,
                           patience = IM_PATIENCE)


    pred_probas = ideology_model_predictor(model, tokens)

    # from the predicted probabilities, we want the second column, which shows the probability
    # of the article being right-wing - a score near to 1 is very right wing; a score near to 0
    # is very left wing. We then add this column onto our df and return the full df.

    df['pred_probas'] = pred_probas[:,1]

    return df


In [ ]:
output_df = full_ideology_model(df)


In [ ]:
output_df.head(10)
